In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
import regex as re

In [26]:
df = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/DOTO_database.csv')

In [3]:
#use regex to remove number from the "Identity" column
df['Identity'] = df['Identity'].str.replace(r'\d+', '')

/var/folders/k8/tcp87zks6m70h7rk_s6t48c40000gn/T/ipykernel_1553/1328613314.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Identity'] = df['Identity'].str.replace(r'\d+', '')


In [5]:
#remove the character "-" from the "Identity" column
df['Identity'] = df['Identity'].str.replace('-', '')

In [6]:
#convert date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df = df[df['State'] != 'Madhya Pradesh']
df = df[df['State'] != 'Delhi']
df = df[df['State'] != 'Jammu and Kashmir']
df = df[df['State'] != 'Chandigarh']
df= df[df['State'] != 'Puducherry']
df = df[df['State'] != 'Andaman and Nicobar Islands']
df = df[df['State'] != 'Lakshadweep']
df = df[df['State'] != 'Dadra and Nagar Haveli']
df = df[df['State'] != 'Daman and Diu']
df= df[df['State'] != 'Jharkhand']
df= df[df['State'] != 'Karnataka']

In [8]:
#create a new column called party_state and if state='Karnataka', 'Gujarat', 'Assam', 'Goa', 'Arunachal Pradesh', 'Haryana', 'Manipur', 'Uttar Pradesh', 'Tripura', 'Uttarakhand', 'Himachal Pradesh', mark BJP. Else Non BJP
df['party'] = np.where(df['State'].isin(['Gujarat', 'Assam', 'Goa', 'Arunachal Pradesh', 'Haryana', 'Manipur', 'Uttar Pradesh', 'Tripura', 'Uttarakhand', 'Himachal Pradesh']), 'BJP', 'Non BJP')

In [9]:
#make a new column called religion
df['religion'] = np.nan

In [10]:
#if the "Identity" column has "Muslims", "Hindu Muslims", "Muslims Christians", "Muslims Hindu", "Muslims Christians", "Muslims Christians Other MInorities", "Muslims Police/Investigating Agency", "Muslims Public servant/Bureaucrat", "Muslims Hindu Police/Investigating Agency", enter "Muslim" in the column
df.loc[(df['Identity'].isin(['Muslims', 'Hindu Muslims', 'Muslims Christians', 'Muslims Hindu', 'Muslims Christians', 'Muslims Christians Other Minorities', 'Muslims Police/Investigating Agency', 'Muslims Public servant/Bureaucrat', 'Muslims Hindu Police/Investigating Agency'])), 'religion'] = 'Muslim'

In [11]:
#if Identity is empty, enter "Other"
df.loc[(df['religion'].isnull()), 'religion'] = 'Other'

In [12]:
#make a new column called year
df['year'] = df['Date'].dt.year

In [13]:
#remove the year 1970
df = df[df['year'] != 1970]

In [14]:
#if a month is between January and April, then it is grouped as month=4, day=30. If it is between May and August,  then it is grouped as month=8, day=31. if it is between September and December, then it is grouped as month=12, day=31
df['Date'] = df['Date'].apply(lambda x: x.replace(month=3, day=31) if x.month <= 3 else x.replace(month=6, day=30) if x.month <= 6 else x.replace(month=9, day=30) if x.month <= 9 else x.replace(month=12, day=31))

In [15]:
#only keep year=2019, 2020, 2021
df = df[df['Date'].dt.year.isin([2019, 2020, 2021])]

In [16]:
#only only rows where religion is "Muslim"
df = df[df['religion'] == 'Muslim']

In [17]:
#count number of incidents by party, date, religion
count_df = df.groupby(['party', 'Date', 'religion']).size().reset_index(name='counts')

In [18]:
#make a column called post with 1 if date is after 1 April 2020 and 0 if date is before 1 April 2020
count_df['post'] = np.where(count_df['Date'] >= '2020-04-01', 1, 0)

In [19]:
#only keep rows where identity is "other"
count_df = count_df[count_df['religion'] == 'Other']

In [ ]:
#make a density plot for pre=0, and party=BJP s
sns.kdeplot(data=count_df[count_df['party'] == 'BJP'][count_df['post'] == 0], x="counts", fill=True, linewidth=1, linestyle='solid', color='blue')
#add a plot for pre=0, party=Non BJP
sns.kdeplot(data=count_df[count_df['party'] == 'Non BJP'][count_df['post'] == 0], x="counts", alpha=.5, fill=True, linewidth=1, linestyle='solid', color='grey')
#add a line for the mean
plt.axvline(count_df[count_df['party'] == 'BJP'][count_df['post'] == 0]['counts'].mean(), color='blue', linestyle='dashed', linewidth=1)
#add a line for the mean
plt.axvline(count_df[count_df['party'] == 'Non BJP'][count_df['post'] == 0]['counts'].mean(), color='grey', linestyle='dashed', linewidth=1)
#add a title in times new roman
plt.title('Density plot of number of incidents per day for BJP and Non BJP before 1 April 2020', fontname='Times New Roman', fontsize=12)
#add a legend
plt.legend(['BJP', 'Non BJP'])

In [ ]:
#make a density plot for post=1, and party=BJP s
sns.kdeplot(data=count_df[count_df['party'] == 'BJP'][count_df['post'] == 1], x="counts", fill=True, linewidth=1, linestyle='solid', color='blue')
#add a plot for post=1, party=Non BJP
sns.kdeplot(data=count_df[count_df['party'] == 'Non BJP'][count_df['post'] == 1], x="counts",fill=True, linewidth=1, linestyle='solid', color='grey')
#add a line for the mean
plt.axvline(count_df[count_df['party'] == 'BJP'][count_df['post'] == 1]['counts'].mean(), color='blue', linestyle='dashed', linewidth=1)
#add a line for the mean
plt.axvline(count_df[count_df['party'] == 'Non BJP'][count_df['post'] == 1]['counts'].mean(), color='grey', linestyle='dashed', linewidth=1)
#add a title in times new roman
plt.title('Density plot of number of incidents per day for BJP and Non BJP after 1 April 2020', fontname='Times New Roman', fontsize=12)
#add a legend
plt.legend(['BJP', 'Non BJP'])


In [ ]:
import plotly.express as px
#plot a line plot with ink blue line for BJP and black line for Non BJP and a plain background
fig = px.line(count_df, x="Date", y="counts", color='party', color_discrete_map={'BJP': 'rgb(0,0,255)', 'Non BJP': 'rgb(0,0,0)'}, template='plotly_white')

#add a vertical line on the date 01-04-2020
fig.add_vline(x="2020-04-01", line_width=1, line_dash="dash", line_color="black")
#label all dates on the x-axis
fig.update_xaxes(
    ticktext=['2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', 
              '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', 
              '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31',],
    tickvals=['2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', 
              '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', 
              '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31'],
    tickangle=90
)

#y-axis label
fig.update_yaxes(title_text='Number of incidents')

#show the plot
fig.show()

In [ ]:
#make density plot of the number of incidents by party in the period before 1 April 2020 and add lines for the mean using plotly
fig = px.density(count_df[count_df['Date'] < '2020-04-01'], x="counts", color="party", template='plotly_white')
fig.add_vline(x=fig.data[0].x.mean(), line_width=1, line_dash="dash", line_color="black")
fig.add_vline(x=fig.data[1].x.mean(), line_width=1, line_dash="dash", line_color="black")
fig.show()

In [65]:
#only keep rows where identity is "Other"
count_df = count_df[count_df['religion'] == 'Other']

In [ ]:
#make dotted line plot with ink blue line for BJP and black line for Non BJP and a plain background for count_df
fig = px.line(count_df, x="Date", y="counts", color='party', color_discrete_map={'BJP': 'rgb(0,0,255)', 'Non BJP': 'rgb(0,0,0)'}, template='plotly_white')
#add a vertical line on the date 01-04-2020
fig.add_vline(x="2020-04-01", line_width=1, line_dash="dash", line_color="black")
#label all dates on the x-axis
fig.update_xaxes(
    ticktext=['2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', 
              '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', 
              '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31',],
    tickvals=['2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', 
              '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', 
              '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31'],
    tickangle=90
)
#y-axis label
fig.update_yaxes(title_text='Number of incidents')
#show the plot
fig.show()

In [154]:
#export df to csv
df.to_csv('/Users/anusharajan/Documents/Assignments_thesis/df_final.csv')

In [299]:
count_data = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/count_data.csv')

In [300]:
#parse the date column as datetime
count_data['Date'] = pd.to_datetime(count_data['Date'], format='%Y-%m-%d')

In [301]:
#count number of rows for each state and make a new df called try_this
try_this = count_data.groupby(['State']).agg({'counts': 'count'}).reset_index()

In [ ]:
#check for duplicates in the date column for every state
for i in count_data['State']:
    print(i)
    print(count_data[count_data['State'] == i]['Date'].duplicated().any())

In [303]:
#make a new column called post and if the date is after 2020-03-24, make the value 1 else 0
count_data['post'] = np.where(count_data['Date'] > '2020-03-24', 1, 0)

In [305]:
#if party is BJP, make the value 1 else 0
count_data['partyid'] = np.where(count_data['party'] == 'BJP', 1, 0)

In [309]:
count_data.to_csv('count_data.csv')

In [16]:
#import cmie df from /Users/anusharajan/Documents/Assignments_thesis/CMIE - Sheet1.csv
cmie = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/CMIE - Sheet1.csv')

In [17]:
#convert it from wide to long
cmie = pd.melt(cmie, id_vars=['States'], value_vars=['Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 
                                                     'Jul 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 
                                                     'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 
                                                     'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 
                                                     'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021', 
                                                     'Jul 2021', 'Aug 2021', 'Sep 2021', 'Oct 2021', 'Nov 2021', 'Dec 2021'])

In [18]:
#convert the variable column to datetime
cmie['variable'] = pd.to_datetime(cmie['variable'], format='%b %Y')

In [19]:
#make a new column called month_year
cmie['month_year'] = cmie['variable'].dt.month.astype(str) + '-' + cmie['variable'].dt.year.astype(str)

In [21]:
#if month_year 1-2020,2-2020, 3-2020 group them together and sum the number of deaths
cmie.loc[cmie['month_year'].isin(['1-2019', '2-2019', '3-2019']), 'month_year'] = '3-2019'
cmie.loc[cmie['month_year'].isin(['4-2019', '5-2019', '6-2019']), 'month_year'] = '6-2019'
cmie.loc[cmie['month_year'].isin(['7-2019', '8-2019', '9-2019']), 'month_year'] = '9-2019'
cmie.loc[cmie['month_year'].isin(['10-2019', '11-2019', '12-2019']), 'month_year'] = '12-2019'


cmie.loc[cmie['month_year'].isin(['1-2020', '2-2020', '3-2020']), 'month_year'] = '3-2020'
cmie.loc[cmie['month_year'].isin(['4-2020', '5-2020', '6-2020']), 'month_year'] = '6-2020'
cmie.loc[cmie['month_year'].isin(['7-2020', '8-2020', '9-2020']), 'month_year'] = '9-2020'
cmie.loc[cmie['month_year'].isin(['10-2020', '11-2020', '12-2020']), 'month_year'] = '12-2020'

cmie.loc[cmie['month_year'].isin(['1-2021', '2-2021', '3-2021']), 'month_year'] = '3-2021'
cmie.loc[cmie['month_year'].isin(['4-2021', '5-2021', '6-2021']), 'month_year'] = '6-2021'
cmie.loc[cmie['month_year'].isin(['7-2021', '8-2021', '9-2021']), 'month_year'] = '9-2021'
cmie.loc[cmie['month_year'].isin(['10-2021', '11-2021', '12-2021']), 'month_year'] = '12-2021'

In [22]:
#group by state and month_year and take average of the value column
cmie = cmie.groupby(['States', 'month_year']).agg({'value': 'mean'}).reset_index()

In [23]:
#convert month_year to datetime
cmie['month_year'] = pd.to_datetime(cmie['month_year'], format='%m-%Y')

In [24]:
#export cmie to csv
cmie.to_csv('cmie.csv')

In [17]:
#import /Users/anusharajan/Documents/Assignments_thesis/count_data.csv
count_data = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/count_data.csv')

In [18]:
#if date is before 1-04-2020, make the value 1 else 0
count_data['pre'] = np.where(count_data['Date'] < '2020-04-01', 0, 1)

In [20]:
#make a column called partyid if BJP, make the value 1 else 0
count_data['partyid'] = np.where(count_data['party'] == 'BJP', 1, 0)

In [12]:
#export to csv
count_data.to_csv('count_data.csv')

In [2]:
covid19 = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/COVID-19 Cases(06-04-2023).csv')

In [3]:
#read date column as datetime
covid19['Date'] = pd.to_datetime(covid19['Date'], format='%d/%m/%Y')

In [4]:
#only keep years 2020 and 2021
covid19 = covid19[covid19['Date'].dt.year.isin([2020, 2021])]

In [5]:
covid19 = covid19[covid19['Region'].isin(['Karnataka', 'Madhya Pradesh', 'Jharkhand'])]

In [6]:
#if a month is between January and April, then it is grouped as month=4, day=30. If it is between May and August,  then it is grouped as month=8, day=31. if it is between September and December, then it is grouped as month=12, day=31
covid19['Date'] = covid19['Date'].apply(lambda x: x.replace(month=3, day=31) if x.month <= 3 else x.replace(month=6, day=30) if x.month <= 6 else x.replace(month=9, day=30) if x.month <= 9 else x.replace(month=12, day=31))
#group by region and date and take the mean of the values
covid19 = covid19.groupby(['Region', 'Date']).agg({'Death': 'mean'}).reset_index()

In [7]:
covid19.to_csv('/Users/anusharajan/Documents/Assignments_thesis/covid_df_robust.csv', index=False)

In [ ]:
state_df = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/OxCGRT_IND_latest.csv')

In [42]:
state_df = state_df[state_df['RegionName'] != 'Madhya Pradesh'] 
state_df = state_df[state_df['RegionName'] != 'Delhi'] 
state_df = state_df[state_df['RegionName'] != 'Jammu and Kashmir'] 
state_df = state_df[state_df['RegionName'] != 'Chandigarh'] 
state_df= state_df[state_df['RegionName'] != 'Puducherry'] 
state_df = state_df[state_df['RegionName'] != 'Andaman and Nicobar Islands'] 
state_df = state_df[state_df['RegionName'] != 'Lakshadweep'] 
state_df = state_df[state_df['RegionName'] != 'Dadra and Nagar Haveli'] 
state_df = state_df[state_df['RegionName'] != 'Daman and Diu'] 
state_df= state_df[state_df['RegionName'] != 'Jharkhand'] 
state_df= state_df[state_df['RegionName'] != 'Karnataka']
state_df= state_df[state_df['RegionName'] != 'Ladakh']

In [43]:
#read Date column as datetime
state_df['Date'] = pd.to_datetime(state_df['Date'], format='%Y%m%d')

In [44]:
#only keep years 2020 and 2021
state_df = state_df[state_df['Date'].dt.year.isin([2020, 2021])]

In [ ]:
#display column names
state_df.columns

In [45]:
state_df = state_df[['RegionName', 'Date', 'C1M_School closing', 'C2M_Workplace closing', 'C3M_Cancel public events', 'C4M_Restrictions on gatherings', 'C5M_Close public transport', 'C6M_Stay at home requirements', 'C7M_Restrictions on internal movement', 'H1_Public information campaigns']]

In [46]:
#remove rows where RegionName=Nan
state_df = state_df[state_df['RegionName'].notna()]

In [47]:
#remove rows with Nan values
state_df = state_df.dropna()

In [48]:
#if a month is between January and April, then it is grouped as month=4, day=30. If it is between May and August,  then it is grouped as month=8, day=31. if it is between September and December, then it is grouped as month=12, day=31
state_df['Date'] = state_df['Date'].apply(lambda x: x.replace(month=3, day=31) if x.month <= 3 else x.replace(month=6, day=30) if x.month <= 6 else x.replace(month=9, day=30) if x.month <= 9 else x.replace(month=12, day=31))

In [49]:
#group by region and date and take the mean of the values
state_df = state_df.groupby(['RegionName', 'Date']).agg({'C1M_School closing': 'mean', 'C2M_Workplace closing': 'mean', 'C3M_Cancel public events': 'mean', 'C4M_Restrictions on gatherings': 'mean', 'C5M_Close public transport': 'mean', 'C6M_Stay at home requirements': 'mean', 'C7M_Restrictions on internal movement': 'mean', 'H1_Public information campaigns': 'mean'}).reset_index()

In [52]:
state_df= state_df[state_df['RegionName'] != 'Dadra and Nagar Haveli and Daman and Diu']

In [3]:
#make partyid column and if BJP, make the value 1 else 0
count_data['partyid'] = np.where(count_data['party'] == 'BJP', 1, 0)
#parse date column as datetime
count_data['Date'] = pd.to_datetime(count_data['Date'], format='%Y-%m-%d')
#make a post column and if date is after 1-04-2020, make the value 1 else 0
count_data['post'] = np.where(count_data['Date'] >= '2020-04-01', 1, 0)

In [10]:
robust_df = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/DOTO_database.csv')

In [11]:
#convert date column to datetime
robust_df['Date'] = pd.to_datetime(robust_df['Date'])
robust_df = robust_df[robust_df['State'] != 'Delhi']
robust_df = robust_df[robust_df['State'] != 'Jammu and Kashmir']
robust_df = robust_df[robust_df['State'] != 'Chandigarh']
robust_df= robust_df[robust_df['State'] != 'Puducherry']
robust_df = robust_df[robust_df['State'] != 'Andaman and Nicobar Islands']
robust_df = robust_df[robust_df['State'] != 'Lakshadweep']
robust_df = robust_df[robust_df['State'] != 'Dadra and Nagar Haveli']
robust_df = robust_df[robust_df['State'] != 'Daman and Diu']


#create a new column called party_state and if state='Karnataka', 'Gujarat', 'Assam', 'Goa', 'Arunachal Pradesh', 'Haryana', 'Manipur', 'Uttar Pradesh', 'Tripura', 'Uttarakhand', 'Himachal Pradesh', mark BJP. Else Non BJP
robust_df['party'] = np.where(robust_df['State'].isin(['Karnataka', 'Madhya Pradesh', 'Gujarat', 'Assam', 'Goa', 'Arunachal Pradesh', 'Haryana', 'Manipur', 'Uttar Pradesh', 'Tripura', 'Uttarakhand', 'Himachal Pradesh']), 'BJP', 'Non BJP')
#make a new column called religion
robust_df['religion'] = np.nan
#if the "Identity" column has "Muslims", "Hindu Muslims", "Muslims Christians", "Muslims Hindu", "Muslims Christians", "Muslims Christians Other MInorities", "Muslims Police/Investigating Agency", "Muslims Public servant/Bureaucrat", "Muslims Hindu Police/Investigating Agency", enter "Muslim" in the column
robust_df.loc[(robust_df['Identity'].isin(['Muslims', 'Hindu Muslims', 'Muslims Christians', 'Muslims Hindu', 'Muslims Christians', 'Muslims Christians Other Minorities', 'Muslims Police/Investigating Agency', 'Muslims Public servant/Bureaucrat', 'Muslims Hindu Police/Investigating Agency'])), 'religion'] = 'Muslim'
#if Identity is empty, enter "Other"
robust_df.loc[(robust_df['religion'].isnull()), 'religion'] = 'Other'
#make a new column called year
robust_df['year'] = robust_df['Date'].dt.year
#remove the year 1970
robust_df = robust_df[robust_df['year'] != 1970]
#if a month is between January and April, then it is grouped as month=4, day=30. If it is between May and August,  then it is grouped as month=8, day=31. if it is between September and December, then it is grouped as month=12, day=31
robust_df['Date'] = robust_df['Date'].apply(lambda x: x.replace(month=3, day=31) if x.month <= 3 else x.replace(month=6, day=30) if x.month <= 6 else x.replace(month=9, day=30) if x.month <= 9 else x.replace(month=12, day=31))
#only keep year=2019, 2020, 2021
robust_df = robust_df[robust_df['Date'].dt.year.isin([2019, 2020, 2021])]
#only only rows where religion is "Muslim"
robust_df = robust_df[robust_df['religion'] == 'Muslim']
#count number of incidents by party, date, religion
count_robust_df = robust_df.groupby(['party', 'Date', 'religion']).size().reset_index(name='counts')
import plotly.express as px
#plot a line plot with ink blue line for BJP and black line for Non BJP and a plain background
fig = px.line(count_robust_df, x="Date", y="counts", color='party', color_discrete_map={'BJP': 'rgb(0,0,255)', 'Non BJP': 'rgb(0,0,0)'}, template='plotly_white')

#add a vertical line on the date 01-04-2020
fig.add_vline(x="2020-04-01", line_width=1, line_dash="dash", line_color="black")
#label all dates on the x-axis
fig.update_xaxes(
    ticktext=['2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', 
              '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', 
              '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31',],
    tickvals=['2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', 
              '2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31', 
              '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31'],
    tickangle=90,
    tickfont=dict(
        size=9,
        color='black')
)

#y-axis label
fig.update_yaxes(title_text='Number of incidents')

#show the plot
fig.show()

In [13]:
#create a new column called counts and count number of incidents by state, party, date, religion
robust_df = robust_df.groupby(['State', 'party', 'Date', 'religion']).size().reset_index(name='counts')

In [14]:
#make a new column called post
robust_df['post'] = np.where(robust_df['Date'] >= '2020-04-01', 1, 0)

In [16]:
#make a column called partyid, if party is BJP, then partyid=1, else partyid=0
robust_df['partyid'] = np.where(robust_df['party'] == 'BJP', 1, 0)

In [8]:
state_df = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/OxCGRT_IND_latest.csv')

In [9]:
#only keep region name = Karnataka, Madhya Pradesh, Jharkhand
state_df = state_df[state_df['RegionName'].isin(['Karnataka', 'Madhya Pradesh', 'Jharkhand'])]

In [10]:
#read Date column as datetime
state_df['Date'] = pd.to_datetime(state_df['Date'], format='%Y%m%d')
#only keep years 2020 and 2021
state_df = state_df[state_df['Date'].dt.year.isin([2020, 2021])]

state_df = state_df[['RegionName', 'Date', 'C1M_School closing', 'C2M_Workplace closing', 'C3M_Cancel public events', 'C4M_Restrictions on gatherings', 'C5M_Close public transport', 'C6M_Stay at home requirements', 'C7M_Restrictions on internal movement', 'H1_Public information campaigns']]
#remove rows where RegionName=Nan
state_df = state_df[state_df['RegionName'].notna()]
#remove rows with Nan values
state_df = state_df.dropna()
#if a month is between January and April, then it is grouped as month=4, day=30. If it is between May and August,  then it is grouped as month=8, day=31. if it is between September and December, then it is grouped as month=12, day=31
state_df['Date'] = state_df['Date'].apply(lambda x: x.replace(month=3, day=31) if x.month <= 3 else x.replace(month=6, day=30) if x.month <= 6 else x.replace(month=9, day=30) if x.month <= 9 else x.replace(month=12, day=31))
#group by region and date and take the mean of the values
state_df = state_df.groupby(['RegionName', 'Date']).agg({'C1M_School closing': 'mean', 'C2M_Workplace closing': 'mean', 'C3M_Cancel public events': 'mean', 'C4M_Restrictions on gatherings': 'mean', 'C5M_Close public transport': 'mean', 'C6M_Stay at home requirements': 'mean', 'C7M_Restrictions on internal movement': 'mean', 'H1_Public information campaigns': 'mean'}).reset_index()


In [11]:
#export the data to a csv file
state_df.to_csv('/Users/anusharajan/Documents/Assignments_thesis/state_df_robust.csv', index=False)

In [8]:
#import /Users/anusharajan/Documents/Assignments_thesis/robust_model.csv
count_final_df = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/robust_model.csv')

In [9]:
#delete rows where State= Karnataka, Madhya Pradesh, Jharkhand
count_final_df = count_final_df[~count_final_df['State'].isin(['Karnataka', 'Madhya Pradesh', 'Jharkhand'])]

In [10]:
#export the data to a csv file
count_final_df.to_csv('/Users/anusharajan/Documents/Assignments_thesis/count_final_df_robust.csv', index=False)

In [4]:
#import /Users/anusharajan/Documents/Assignments_thesis/working_file.csv
working_df = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/working_file.csv')

In [5]:
#if blame = "action", blame_number = 4
working_df['blame_number'] = np.where(working_df['blame'] == 'action', 4, 0)
#if blame = "acc_in_threat", blame_number=5
working_df['blame_number'] = np.where(working_df['blame'] == 'acc_in_threat', 5, working_df['blame_number'])
#if blame = "ct_st_ho", blame_number= 2
working_df['blame_number'] = np.where(working_df['blame'] == 'ct_st_ho', 2, working_df['blame_number'])
#if blame = "support", blame_number= 1
working_df['blame_number'] = np.where(working_df['blame'] == 'support', 1, working_df['blame_number'])
#if blame = "blame", blame_number= 3
working_df['blame_number'] = np.where(working_df['blame'] == 'blame', 3, working_df['blame_number'])

In [6]:
#if blame_number=3,4,5 then blame=1, else blame=0
working_df['blame'] = np.where(working_df['blame_number'].isin([3, 4, 5]), 1, 0)

In [ ]:
#export the data to a csv file as final_mechanism.csv
working_df.to_csv('/Users/anusharajan/Documents/Assignments_thesis/final_mechanism.csv', index=False)

In [7]:
#import /Users/anusharajan/Documents/Assignments_thesis/for_analysis.csv
analysis_df = pd.read_csv('/Users/anusharajan/Documents/Assignments_thesis/for_analysis.csv')

In [8]:
#only keep rows where region=center
analysis_df = analysis_df[analysis_df['region'].isin(['centre'])]

In [16]:
#count number of statements by blame_number and store it as a dataframe
analysis_df = analysis_df.groupby(['blame_number']).size().reset_index(name='counts')

In [20]:
#total number of statements
total = analysis_df['counts'].sum()